In [1]:
import pandas as pd
import json

def join_csv_to_geojson(csv_path, geojson_path, output_geojson_path):
    # Load CSV data
    csv_data = pd.read_csv(csv_path)
    
    # Load GeoJSON data
    with open(geojson_path, 'r') as f:
        geojson_data = json.load(f)
    
    # Create a dictionary for quick lookup of CSV data by study_area_id
    csv_dict = {}
    for _, row in csv_data.iterrows():
        study_area_id = int(row['study_area_id'])  # Ensure ID is integer
        if study_area_id not in csv_dict:
            csv_dict[study_area_id] = []
        csv_dict[study_area_id].append({
            'date': row['date'],
            'predicted_hotspot_count': row['predicted_hotspot_count']
        })
    
    # Iterate through GeoJSON features and add CSV data to properties
    for feature in geojson_data['features']:
        feature_id = int(feature['properties']['id'])  # Ensure ID is integer
        if feature_id in csv_dict:
            # Add CSV data as a list of predictions under 'predictions' key
            feature['properties']['predictions'] = csv_dict[feature_id]
    
    # Save updated GeoJSON to a new file
    with open(output_geojson_path, 'w') as f:
        json.dump(geojson_data, f, indent=2)

# Example usage
csv_path = 'new_area_predictions.csv'
geojson_path = 'hex_forest_pro_4326.geojson'
output_geojson_path = 'hex_forest_pro_4326_with_predictions.geojson'

join_csv_to_geojson("predicted_hotspot.csv", "hex_forest_pro_4326.geojson", "hex_forest_pro_4326_with_predictions.geojson")

In [1]:
import json
from shapely.geometry import shape, mapping

def remove_duplicate_polygons(geojson_path, output_geojson_path):
    # Load GeoJSON data
    with open(geojson_path, 'r') as f:
        geojson_data = json.load(f)
    
    # List to store unique features
    unique_features = []
    seen_geometries = set()
    
    # Iterate through features
    for feature in geojson_data['features']:
        # Convert GeoJSON geometry to Shapely geometry
        geom = shape(feature['geometry'])
        
        # Convert geometry to a sorted tuple of coordinates for hashing
        geom_tuple = tuple(sorted(mapping(geom)['coordinates']))
        
        # Check if geometry is already seen
        if geom_tuple not in seen_geometries:
            seen_geometries.add(geom_tuple)
            unique_features.append(feature)
    
    # Update GeoJSON with unique features
    geojson_data['features'] = unique_features
    
    # Save updated GeoJSON to a new file
    with open(output_geojson_path, 'w') as f:
        json.dump(geojson_data, f, indent=2)
    
    print(f"Removed {len(geojson_data['features']) - len(unique_features)} duplicate polygons.")
    print(f"Saved unique features to {output_geojson_path}")

# Example usage
geojson_path = 'hex_forest_pro_4326_predict_old.geojson'
output_geojson_path = 'hex_forest_pro_4326_predict.geojson'

remove_duplicate_polygons(geojson_path, output_geojson_path)

Removed 0 duplicate polygons.
Saved unique features to hex_forest_pro_4326_predict.geojson
